In [1]:
import pandas as pd
import pyomo.environ as pe
import math
# ============
# 0) DATA LOAD
# ============
customers_df = pd.read_csv("customers.csv")
loc_df = pd.read_csv("locations.csv")
params_df = pd.read_csv("params.csv")

Q = float(params_df.loc[0, "Q"])
p = float(params_df.loc[0, "p"])
c_km = float(params_df.loc[0, "c"])

C = customers_df["customer"].astype(str).tolist()
T = ["M", "A"]
N = ["0"] + C

d = {row["customer"]: float(row["demand"]) for _, row in customers_df.iterrows()}
a = {(row["customer"], "M"): int(row["avail_M"]) for _, row in customers_df.iterrows()}
a.update({(row["customer"], "A"): int(row["avail_A"]) for _, row in customers_df.iterrows()})


coords = {
    str(row["node"]): (float(row["x"]), float(row["y"]))
    for _, row in loc_df.iterrows()
}

N = list(coords.keys())  # incluye '0' y clientes

# Distancias euclídeas completas
dist = {}
for i in N:
    for j in N:
        if i != j:
            xi, yi = coords[i]
            xj, yj = coords[j]
            dist[(i, j)] = math.hypot(xi - xj, yi - yj)


# =================
# 1) PYOMO MODEL
# =================
m = pe.ConcreteModel()

# 1) SETS
m.C = pe.Set(initialize=C)
m.T = pe.Set(initialize=T)
m.N = pe.Set(initialize=N)

# 2) PARAMETERS
m.d = pe.Param(m.C, initialize=d)
m.a = pe.Param(m.C, m.T, initialize=a)
m.dist = pe.Param(m.N, m.N, initialize=lambda mm, i, j: dist[(i, j)] if i != j else 0.0, default=0.0)

# 3) VARIABLES
m.x = pe.Var(m.N, m.N, m.T, within=pe.Binary)
m.served = pe.Var(m.C, m.T, within=pe.Binary)
m.u = pe.Var(m.C, m.T, within=pe.NonNegativeReals)

# 4) OBJECTIVE
def obj_rule(mm):
    revenue = sum(p * mm.d[i] * mm.served[i, t] for i in mm.C for t in mm.T)
    travel = sum(c_km * mm.dist[i, j] * mm.x[i, j, t] for i in mm.N for j in mm.N if i != j for t in mm.T)
    return revenue - travel

m.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)

# 5) CONSTRAINTS

# 5.0 No self arcs
def no_self_rule(mm, i, t):
    return mm.x[i, i, t] == 0
m.no_self = pe.Constraint(m.N, m.T, rule=no_self_rule)

# 5.1 Each customer served exactly once during the day
def assign_rule(mm, i):
    return sum(mm.served[i, t] for t in mm.T) == 1
m.assign = pe.Constraint(m.C, rule=assign_rule)

# 5.2 Time availability
def avail_rule(mm, i, t):
    return mm.served[i, t] <= mm.a[i, t]
m.avail = pe.Constraint(m.C, m.T, rule=avail_rule)

# 5.3 Flow conservation if served: 1 in, 1 out
def out_flow_rule(mm, i, t):
    return sum(mm.x[i, j, t] for j in mm.N if j != i) == mm.served[i, t]
m.out_flow = pe.Constraint(m.C, m.T, rule=out_flow_rule)

def in_flow_rule(mm, i, t):
    return sum(mm.x[j, i, t] for j in mm.N if j != i) == mm.served[i, t]
m.in_flow = pe.Constraint(m.C, m.T, rule=in_flow_rule)

# 5.4 Depot departure and return (one route per turn)
def depot_depart_rule(mm, t):
    return sum(mm.x["0", j, t] for j in mm.C) == 1
m.depot_depart = pe.Constraint(m.T, rule=depot_depart_rule)

def depot_return_rule(mm, t):
    return sum(mm.x[i, "0", t] for i in mm.C) == 1
m.depot_return = pe.Constraint(m.T, rule=depot_return_rule)

# 5.5 Capacity per trip
def cap_rule(mm, t):
    return sum(mm.d[i] * mm.served[i, t] for i in mm.C) <= Q
m.capacity = pe.Constraint(m.T, rule=cap_rule)

# 5.6 MTZ subtour elimination (per turn)
M = len(C)

def mtz_rule(mm, i, j, t):
    if i == j:
        return pe.Constraint.Skip
    return mm.u[i, t] - mm.u[j, t] + M * mm.x[i, j, t] <= M - 1
m.mtz = pe.Constraint(m.C, m.C, m.T, rule=mtz_rule)

def u_upper_rule(mm, i, t):
    return mm.u[i, t] <= M * mm.served[i, t]
m.u_upper = pe.Constraint(m.C, m.T, rule=u_upper_rule)

def u_lower_rule(mm, i, t):
    return mm.u[i, t] >= mm.served[i, t]
m.u_lower = pe.Constraint(m.C, m.T, rule=u_lower_rule)

# =================
# 6) SOLVE (GUROBI)
# =================
solver = pe.SolverFactory("gurobi")
res = solver.solve(m, tee=False)

# =================
# 7) RESULTS
# =================
def extract_route(turn):
    nxt = {}
    for i in N:
        for j in N:
            if i != j and pe.value(m.x[i, j, turn]) > 0.5:
                nxt[i] = j
                break
    route = ["0"]
    seen = set(["0"])
    while True:
        cur = route[-1]
        if cur not in nxt:
            break
        route.append(nxt[cur])
        if route[-1] == "0":
            break
        if route[-1] in seen:
            break
        seen.add(route[-1])
    return route

def route_distance(route, turn):
    total = 0.0
    for k in range(len(route) - 1):
        total += dist[(route[k], route[k+1])]
    return total

print("\n==== MODELO 1 RESULTADOS ====")
obj = pe.value(m.obj)
print(f"Beneficio diario (obj): {obj:.4f} €")

total_revenue = sum(p * d[i] * pe.value(m.served[i, t]) for i in C for t in T)
total_km_cost = sum(c_km * dist[(i, j)] * pe.value(m.x[i, j, t]) for i in N for j in N if i != j for t in T)
print(f"Ingresos: {total_revenue:.4f} €")
print(f"Coste km: {total_km_cost:.4f} €")

for t in T:
    served_list = [i for i in C if pe.value(m.served[i, t]) > 0.5]
    load = sum(d[i] for i in served_list)
    route = extract_route(t)
    km = route_distance(route, t)
    print(f"\nTurno {t}:")
    print(f"  Clientes: {', '.join(served_list) if served_list else '-'}")
    print(f"  Carga: {load:.2f} / {Q:.2f}")
    print(f"  Ruta: {' -> '.join(route)}")
    print(f"  Distancia: {km:.2f} km")



==== MODELO 1 RESULTADOS ====
Beneficio diario (obj): 1343.2709 €
Ingresos: 1405.0000 €
Coste km: 61.7291 €

Turno M:
  Clientes: C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C25, C26
  Carga: 139.00 / 180.00
  Ruta: 0 -> C7 -> C26 -> C10 -> C3 -> C8 -> C5 -> C12 -> C1 -> C9 -> C4 -> C2 -> C11 -> C25 -> C6 -> 0
  Distancia: 76.36 km

Turno A:
  Clientes: C13, C14, C15, C16, C17, C18, C19, C20, C21, C22, C23, C24, C27, C28
  Carga: 142.00 / 180.00
  Ruta: 0 -> C18 -> C15 -> C20 -> C23 -> C14 -> C21 -> C28 -> C19 -> C24 -> C17 -> C16 -> C27 -> C13 -> C22 -> 0
  Distancia: 77.97 km
